In [ ]:
from langchain_docling import DoclingLoader

In [ ]:
sourece = "ReAct.pdf"
loader = DoclingLoader(
    file_path=sourece,
    export_type = "markdown"
)
documents = loader.load()

In [ ]:
print(documents[0].page_content[:500])  # Print the first 500 characters of the first document

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Docling 마크다운 출력용 섹션 단위 분할
text_splitter = RecursiveCharacterTextSplitter(
    # 청크 최대 크기: 1200자
    # - 영어 논문 1200자 ≈ 300토큰 (대부분 임베딩 모델 512토큰 제한 내)
    # - 논문 섹션(Abstract, Introduction) 전체를 한 청크로 유지 가능
    # - 너무 작으면 문맥 손실, 너무 크면 검색 정확도 하락
    chunk_size=1200,
    
    # 청크 간 겹치는 부분: 200자 (chunk_size의 16%)
    # - 섹션 경계에서 문맥이 끊기는 것 방지
    # - 검색 시 경계 부근 정보 놓치지 않도록
    # - 일반적으로 chunk_size의 10-20% 권장
    chunk_overlap=200,
    
    # 분할 우선순위 (위에서부터 순서대로 시도)
    separators=[
        "\n## ",      # 1순위: 논문 섹션 헤더 (## Abstract, ## Introduction)
                      # Docling이 주요 섹션을 2레벨 헤더로 변환하므로
                      # 가장 큰 의미 단위인 섹션을 먼저 보존
        
        "\n### ",     # 2순위: 서브섹션 (### 3.1 Dataset, ### 3.2 Model)
                      # 섹션이 1200자 초과 시 서브섹션 단위로 자연스럽게 분할
        
        "\n\n",       # 3순위: 문단 경계
                      # 서브섹션도 길면 문단 단위로 분할
                      # 논문은 문단이 논리적 단위이므로 여기서 자르는 게 자연스러움
        
        "\n",         # 4순위: 줄바꿈
                      # 문단도 길면 줄 단위로
                      # 수식, 리스트, 코드 블록 등은 줄바꿈으로 구분됨
        
        ". ",         # 5순위: 문장 끝 (마침표 + 공백)
                      # 최소한 완전한 문장은 유지
                      # 공백까지 포함해야 문장 끝 정확히 감지
        
        " ",          # 6순위: 단어 경계 (공백)
                      # 문장도 초과하면 단어 단위로
                      # 최소한 단어는 쪼개지지 않게
        
        ""            # 7순위: 강제 분할 (문자 단위)
                      # 긴 URL, 코드, 수식 등 chunk_size 초과 시 문자로 강제 분할
                      # 에러 방지용 최후의 안전장치
    ],
    
    # 원본 문서에서의 시작 위치를 메타데이터에 추가
    # - RAG 검색 후 출처 추적 가능 (몇 번째 문자/페이지에서 왔는지)
    # - chunk.metadata['start_index']로 접근
    # - 사용자에게 "논문 3페이지 Introduction 섹션" 같은 정보 제공 가능
    add_start_index=True,
)
chunks = text_splitter.split_documents(documents)

In [ ]:
print(chunks[2])

In [ ]:
from FlagEmbedding import BGEM3FlagModel

In [ ]:
embedding_model = BGEM3FlagModel("BAAI/bge-m3")
embedding_model.model.to("cuda")
embedding_model.model.half()

In [ ]:
print(next(embedding_model.model.parameters()).device)
print(next(embedding_model.model.parameters()).dtype)

In [ ]:
texts = [chunk.page_content for chunk in chunks]

In [ ]:
print(texts)

In [ ]:
len(texts)

In [ ]:
outputs = embedding_model.encode(
    texts,
    batch_size = 12,
    max_length = 8192,
    return_dense = True,
    return_sparse = True,
    return_colbert_vecs=False,
)

In [ ]:
outputs['dense_vecs'][0][:5]

In [ ]:
list(outputs['lexical_weights'][0].items())[:5] 

In [ ]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_compress=True,
    use_ssl=False,
    verify_certs=False
)

# 연결 확인
info = client.info()
print(f"OpenSearch version: {info['version']['number']}")


In [ ]:
reranker_model_id = "n5QFAJwBOacQ-9pbxjwZ"

In [ ]:
index_name = "papers"

index_body = {
    "settings": {
        "index": {
            # k-NN 검색 활성화 (벡터 유사도 검색 사용)
            "knn": True,
            
            # HNSW 검색 시 탐색할 이웃 노드 수
            # 높을수록 정확하지만 느림 (기본 100, 범위 1~10000)
            # 논문 검색은 정확도가 중요해서 100 설정
            "knn.algo_param.ef_search": 100
        }
    },
    "mappings": {
        "properties": {
            # 논문 본문/제목 등 텍스트 저장
            "text": {
                "type": "text",  # 전문 검색 가능한 텍스트 필드
                "analyzer": "standard"  # 영어 기본 분석기 (토큰화, 소문자화)
            },
            
            # 논문 메타데이터 (저자, 날짜, 출처 등)
            "metadata": {
                "type": "object",  # JSON 객체 저장
                "enabled": True  # 인덱싱 활성화 (검색/필터링 가능)
            },
            
            # BGE-M3 dense vector (의미 기반 검색)
            "dense_vector": {
                "type": "knn_vector",  # 벡터 검색용 필드
                "dimension": 1024,  # BGE-M3 출력 차원
                
                "method": {
                    # HNSW: 근사 최근접 이웃 알고리즘 (빠른 벡터 검색)
                    "name": "hnsw",
                    
                    # cosinesimil: 코사인 유사도 (방향 기반 유사도)
                    # 논문 검색은 의미 유사도가 중요해서 cosine 선택
                    # 대안: l2(유클리드), innerproduct(내적)
                    "space_type": "cosinesimil",
                    
                    # faiss: Meta의 벡터 검색 라이브러리 (빠르고 메모리 효율적)
                    # 대안: lucene(작은 데이터), nmslib(구식)
                    "engine": "faiss",
                    
                    "parameters": {
                        # ef_construction: 인덱스 구축 시 탐색 범위
                        # 높을수록 정확한 그래프 구축, 느린 인덱싱 (기본 100)
                        # 128: 논문처럼 고품질 검색이 필요한 경우
                        "ef_construction": 128,
                        
                        # m: 각 노드가 연결할 이웃 수
                        # 높을수록 정확하지만 메모리/속도 trade-off (기본 16)
                        # 16: 균형잡힌 설정 (범위 2~100)
                        "m": 16
                    }
                }
            },
            
            # BGE-M3 sparse vector (키워드 기반 검색)
            "sparse_vector": {
                # rank_features: 토큰별 가중치 저장 (BM25와 유사)
                # {"machine": 0.8, "learning": 0.6, ...} 형태
                # dense와 결합해서 hybrid search 구현
                "type": "rank_features"
            }
        }
    }
}

In [ ]:
# 인덱스 존재 확인 및 생성
if not client.indices.exists(index=index_name):  # index= 키워드 필수
    client.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created")
else:
    print(f"Index '{index_name}' already exists")

In [ ]:
from opensearchpy import helpers
from tqdm import tqdm

# bulk 색인용 액션 리스트 생성
actions = []
for i, chunk in enumerate(tqdm(chunks, desc="Indexing documents")):
    action = {
        # 인덱스 이름 지정
        "_index": index_name,
        
        # 문서 ID (문자열 권장)
        "_id": str(i),
        
        # 문서 내용 (_body 아님!)
        "_source": {
            "text": chunk.page_content,
            "metadata": chunk.metadata,
            
            # BGE-M3 dense vector (의미 기반 검색)
            # .tolist(): numpy array → Python list 변환 (JSON 직렬화)
            "dense_vector": outputs['dense_vecs'][i].tolist(),
            
            # BGE-M3 sparse vector (키워드 기반 검색)
            # rank_features 타입은 {"token": weight, ...} dict 형태 필요
            "sparse_vector": outputs['lexical_weights'][i]
        }
    }
    actions.append(action)

In [ ]:
# helpers.bulk로 대량 색인
success, failed = helpers.bulk(
    client,  # OpenSearch 클라이언트
    actions,  # 색인할 문서 리스트
    
    # 한 번에 보낼 문서 수 (메모리/성능 균형)
    chunk_size=500,
    
    # 요청 타임아웃 (초)
    request_timeout=60,
    
    # 에러 발생 시 중단하지 않고 계속 진행
    raise_on_error=False
)

print(f"Successfully indexed {success} documents, failed to index {failed} documents")

In [ ]:
# Search Pipeline 생성 (RRF 방식)
# 출처: https://opensearch.org/docs/latest/search-plugins/search-pipelines/score-ranker-processor/
response = client.transport.perform_request(
    'PUT',
    '/_search/pipeline/hybrid_rrf_rerank',
    body={
        "description": "Hybrid search with RRF and Cross-Encoder Reranking",

        # ===== 1단계: RRF (Reciprocal Rank Fusion) =====
        # Dense/Sparse 결과를 순위 기반으로 병합
        "phase_results_processors": [
            {
                "score-ranker-processor": {
                    "combination": {
                        # RRF: 순위 기반 병합 (점수 스케일 무관)
                        "technique": "rrf",

                        # rank_constant: RRF 공식의 k 값 (기본 60)
                        # 공식: score = 1 / (k + rank)
                        "rank_constant": 60,

                        # weights: [dense, sparse] 가중치
                        "parameters": {
                            "weights": [0.5, 0.5]  # 균형 설정
                        }
                    }
                }
            }
        ],

        # ===== 2단계: Cross-Encoder Reranking =====
        # RRF 결과를 ML 모델로 정밀 재평가
        "response_processors": [
            {
                "rerank": {
                    "ml_opensearch": {
                        "model_id": reranker_model_id  # 등록한 모델 ID
                    },
                    "context": {
                        "document_fields": ["text"]  # 분석 대상 필드
                    }
                }
            }
        ]
    }
)

print("✅ Search Pipeline 'hybrid_rrf_rerank' created successfully!")


In [ ]:
# Search Pipeline 조회: 생성된 Pipeline 설정 확인

# HTTP GET 요청으로 특정 Pipeline의 설정을 가져옴
pipeline = client.transport.perform_request(
    'GET',
    '/_search/pipeline/hybrid_rrf_rerank'  # 생성한 파이프라인 이름
)

# 반환값 출력
# - Pipeline이 존재하면: 설정 내용 (dict 형태)
# - Pipeline이 없으면: 404 에러 발생
print(pipeline)


In [ ]:
def search_papers(query, top_k=5):
    """
    Hybrid Search 함수: Dense + Sparse + Reranking
    
    """

    # ============================================
    # 1. 쿼리 인코딩 (BGE-M3)
    # ============================================
    query_output = embedding_model.encode(
        [query],
        return_dense=True,   # 1024차원 벡터
        return_sparse=True   # 단어:가중치 딕셔너리
    )

    # ============================================
    # 2. Hybrid Search 쿼리 구성
    # ============================================
    search_body = {
        "size": top_k * 10,  # Reranker용으로 더 많이 가져오기
        "_source": ["text", "metadata"],  # 벡터 필드 제외

        "query": {
            "hybrid": {
                "queries": [
                    # Dense 검색 (의미 기반)
                    {
                        "knn": {
                            "dense_vector": {
                                "vector": query_output['dense_vecs'][0].tolist(),
                                "k": top_k * 10
                            }
                        }
                    },

                    # Sparse 검색 (키워드 기반)
                    {
                        "bool": {
                            "should": [
                                {
                                    "rank_feature": {
                                        "field": f"sparse_vector.{word}",
                                        "boost": score
                                    }
                                }
                                for word, score in query_output['lexical_weights'][0].items()
                            ]
                        }
                    }
                ]
            }
        },

        # Rerank Extension (Reranker에 쿼리 전달)
        "ext": {
            "rerank": {
                "query_context": {
                    "query_text": query  # Cross-Encoder가 사용
                }
            }
        }
    }

    # ============================================
    # 3. 검색 실행 (Search Pipeline 적용)
    # ============================================
    response = client.search(
        index=index_name,
        body=search_body,
        params={"search_pipeline": "hybrid_rrf_rerank"}
    )

    # ============================================
    # 4. 결과 파싱 (상위 top_k개만)
    # ============================================
    results = []
    for hit in response["hits"]["hits"][:top_k]:
        results.append({
            "text": hit["_source"]["text"],
            "metadata": hit["_source"]["metadata"],
            "score": hit["_score"]  # RRF + Reranker 최종 점수
        })

    return results


In [ ]:
# ============================================
# Hybrid Search 실행 예시
# ============================================

query = "ReAct vs Chain-of-Thought prompting"
results = search_papers(query, top_k=3)

print(f"\n🔍 검색어: '{query}'")
print(f"📊 결과: {len(results)}개\n")

# 결과가 없을 경우 처리
if not results:
    print("❌ 검색 결과가 없습니다.")
else:
    for i, result in enumerate(results, 1):
        print(f"{'='*80}")
        print(f"📄 결과 {i} | 점수: {result['score']:.4f}")
        print(f"{'='*80}")

        # 텍스트 (처음 300자만 표시)
        text = result['text']
        display_text = text[:300] + "..." if len(text) > 300 else text
        print(f"텍스트:\n{display_text}")

        # 메타데이터 출력
        print(f"\n메타데이터:")
        metadata = result.get('metadata', {})
        if metadata:
            for key, value in metadata.items():
                print(f"  - {key}: {value}")
        else:
            print("  (메타데이터 없음)")

        print()